### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import CMNISTDataset
from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

cmnist = CMNISTDataset(
    data_root=CFG["datasets"]["CMNIST"]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = "CMNIST" 
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-11 02:03:40,804 | INFO | Initialized experiment directories for CMNIST
2025-10-11 02:03:40,804 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\CMNIST\logs
2025-10-11 02:03:40,804 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\CMNIST\checkpoints
2025-10-11 02:03:40,804 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\CMNIST\plots


### Setup

In [4]:
domains = CFG["datasets"]["CMNIST"]["domains"]
loaders = {d: {"train": cmnist.get_dataloader(d, train=True), "val": cmnist.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset="CMNIST")
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key="CMNIST",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-11 02:03:46,181 | INFO | === LODO: Leaving out domain 'red' ===



=== LODO: Leaving out domain 'red' ===


Evaluating: 100%|██████████| 50/50 [00:10<00:00,  4.67it/s]
2025-10-11 02:04:12,782 | INFO | [red] Epoch 1/5 | Train - Loss: 0.7781, Cls: 0.7701, GRQO: 0.0080, Acc: 0.7615 | Val - Loss: 0.2058, Cls: 0.2049, GRQO: 0.0008, Acc: 0.9353
2025-10-11 02:04:12,827 | INFO | [red] New best val acc: 0.9353


[red] Epoch 1/5 | Train - Loss: 0.7781, Cls: 0.7701, GRQO: 0.0080, Acc: 0.7615 | Val - Loss: 0.2058, Cls: 0.2049, GRQO: 0.0008, Acc: 0.9353
[red] New best val acc: 0.9353


Evaluating: 100%|██████████| 50/50 [00:10<00:00,  4.87it/s]
2025-10-11 02:04:38,472 | INFO | [red] Epoch 2/5 | Train - Loss: 0.0932, Cls: 0.0901, GRQO: 0.0030, Acc: 0.9770 | Val - Loss: 0.0897, Cls: 0.0892, GRQO: 0.0005, Acc: 0.9749
2025-10-11 02:04:38,516 | INFO | [red] New best val acc: 0.9749


...

[blue] Epoch 3/5 | Train - Loss: 0.0449, Cls: 0.0423, GRQO: 0.0026, Acc: 0.9881 | Val - Loss: 0.0974, Cls: 0.0972, GRQO: 0.0002, Acc: 0.9700


Evaluating: 100%|██████████| 49/49 [00:10<00:00,  4.73it/s]
2025-10-11 02:09:50,500 | INFO | [blue] Epoch 4/5 | Train - Loss: 0.0319, Cls: 0.0297, GRQO: 0.0022, Acc: 0.9928 | Val - Loss: 0.1429, Cls: 0.1428, GRQO: 0.0001, Acc: 0.9561


[blue] Epoch 4/5 | Train - Loss: 0.0319, Cls: 0.0297, GRQO: 0.0022, Acc: 0.9928 | Val - Loss: 0.1429, Cls: 0.1428, GRQO: 0.0001, Acc: 0.9561


Evaluating: 100%|██████████| 49/49 [00:10<00:00,  4.75it/s]
2025-10-11 02:10:16,064 | INFO | [blue] Epoch 5/5 | Train - Loss: 0.0364, Cls: 0.0345, GRQO: 0.0019, Acc: 0.9912 | Val - Loss: 0.0802, Cls: 0.0801, GRQO: 0.0001, Acc: 0.9758
2025-10-11 02:10:16,126 | INFO | [blue] New best val acc: 0.9758
2025-10-11 02:10:16,126 | INFO | [blue] Best Acc: 0.9758
2025-10-11 02:10:16,126 | INFO | ------------------------------------------------------------
2025-10-11 02:10:16,126 | INFO | LODO finished | Mean Acc: 0.9754 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\CMNIST\logs\lodo_summary_20251011_021016.json


[blue] Epoch 5/5 | Train - Loss: 0.0364, Cls: 0.0345, GRQO: 0.0019, Acc: 0.9912 | Val - Loss: 0.0802, Cls: 0.0801, GRQO: 0.0001, Acc: 0.9758
[blue] New best val acc: 0.9758
[blue] Best Acc: 0.9758
------------------------------------------------------------
LODO finished | Mean Acc: 0.9754
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\CMNIST\logs\lodo_summary_20251011_021016.json


### Baseline

In [6]:
model_name = "WinKawaks/vit-tiny-patch16-224"
baseline_results, baseline_mean = run_baseline(
    model_name=model_name,
    CFG=CFG,
    logger=logger,
    dataset_key="CMNIST",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-11 02:10:16,137 | INFO | Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([10]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([10, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-10-11 02:10:17,144 | INFO | === Baseline LODO: Leaving out domain 'red' ===



=== Baseline LODO: Leaving out domain 'red' ===


2025-10-11 02:10:40,714 | INFO | [red] Epoch 1/5 | Train - Loss: 0.5103, Acc: 0.8295 | Val Acc: 0.9495


...

[blue] Epoch 3/5 | Train - Loss: 0.0339, Acc: 0.9890 | Val Acc: 0.9713


2025-10-11 02:15:51,681 | INFO | [blue] Epoch 4/5 | Train - Loss: 0.0213, Acc: 0.9933 | Val Acc: 0.9695


[blue] Epoch 4/5 | Train - Loss: 0.0213, Acc: 0.9933 | Val Acc: 0.9695


2025-10-11 02:16:15,663 | INFO | [blue] Epoch 5/5 | Train - Loss: 0.0183, Acc: 0.9943 | Val Acc: 0.9666
2025-10-11 02:16:15,663 | INFO | [blue] Best Val Acc: 0.9713
2025-10-11 02:16:15,663 | INFO | ------------------------------------------------------------
2025-10-11 02:16:15,663 | INFO | Baseline LODO (WinKawaks/vit-tiny-patch16-224) finished | Mean Acc: 0.9670


[blue] Epoch 5/5 | Train - Loss: 0.0183, Acc: 0.9943 | Val Acc: 0.9666
[blue] Best Val Acc: 0.9713
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-tiny-patch16-224) finished | Mean Acc: 0.9670
